                                          1.Extracting CEO 

In [1]:
# import important libraries

In [1]:
import re
import glob
import nltk

In [2]:
#load into dataset and get the corpus

In [3]:
ceo_path = 'all/ceo.csv'
percent_path = 'all/percentage.csv'
company_path = 'all/companies.csv'

In [4]:
def process_ceo (name):
    
    name = name.strip().split(',')
    name = filter(None, name)
    name = list(map(lambda x: x.strip(),name))
    if len(name) == 0:
        return False
    elif len(name) == 1:
        return name[0]
    elif len(name) == 2:
        return name[0] + ' ' + name[1]
    else:
        return name

In [5]:
ceo_file = open(ceo_path,'r', encoding = 'latin-1').readlines()
ceo_name = map(lambda x: process_ceo(x), ceo_file)
ceo_name = list(set(ceo_name))

In [6]:
def process_percent(percent):
    
    percent = percent.strip().replace('"','').split(' ')
    percent = filter(None, percent)
    percent = list(percent)
    
    if len(percent) == 0:
        return False
    elif len(percent) == 1:
        return percent[0]
    elif len(percent) >= 2:
        return percent[0] + ' ' + percent[1]

In [7]:
percent_file = open(percent_path, 'r', encoding = 'latin-1').readlines()
percent = map(lambda x: process_percent(x), percent_file)
percent = list(set(percent))

In [8]:
def process_company(company):
    
    company = company.strip()
    
    if len(company) == 0:
        return False
    else:
        return company

In [9]:
company_file = open(company_path, 'r', encoding ='latin-1').readlines()
companies = map(lambda x: process_company(x), company_file)
companies = list(set(companies))

In [10]:
# load into corpus 

In [11]:
path_2013 = '2013'
path_2014 = '2014'
path_ceo ='all/ceo.csv'
path_percent = 'all/percentage.csv'
path_companies = 'all/companies.csv'

In [12]:
from nltk.tokenize import sent_tokenize

In [13]:
def get_files(dir):
    files = glob.glob(dir + '/*')
    return files

In [14]:
files_2013 = get_files(path_2013)
files_2014 = get_files(path_2014)
files = files_2013+files_2014

In [15]:
import itertools

In [16]:
def get_corpus(files):
    corpus =[]
    for i, file in enumerate(files):
        with open(file, 'r',encoding = 'latin-1') as f:
            articles =[]
            for article in f:
                article = sent_tokenize(article)
                articles.append(article)
        corpus.append(articles)
        
        if i % 100 ==0:
            print(f'have already read {i} in total')
    
    with open('all_news.txt','w') as an:
        an.write(str(corpus))
    return 

In [17]:
get_corpus(files)

have already read 0 in total
have already read 100 in total
have already read 200 in total
have already read 300 in total
have already read 400 in total
have already read 500 in total
have already read 600 in total
have already read 700 in total


In [18]:
import ast

In [19]:
with open('all_news.txt','r') as an:
    corpus = ast.literal_eval(an.read())
assert len(corpus) == 730

In [20]:
corpus = list(itertools.chain(*corpus))

In [23]:
print(len(corpus))
split_first = int(0.6 * len(corpus))
split_second = int(0.8 * len(corpus))

35898


In [23]:
training_set = corpus[0:split_first]
validating_set = corpus[split_first:split_second]
testing_set = corpus[split_second:]
assert len(corpus) == len(training_set) + len(validating_set) + len(testing_set)

In [24]:
with open('training_final.txt', 'w') as tf:
    tf.write(str(training_set))
with open('validating_final.txt','w') as vf:
    vf.write(str(validating_set))
with open('testing_final.txt','w') as tef:
    tef.write(str(testing_set))
with open('train_set.txt','r') as tf:
    train_set = ast.literal_eval(tf.read())
with open('validate_set.txt','r') as vf:
    valid_set = ast.literal_eval(vf.read())
with open('test_set.txt','r') as tef:
    test_set = ast.literal_eval(tef.read())

In [25]:
from nltk.corpus import stopwords

In [26]:
stop_words = stopwords.words('english')

In [27]:
def check_sw(name):
    for word in stop_words:
        if word in name.lower().split(): 
            return False

    return True

In [28]:
def check_notceo(name):
    first_name, last_name = name.split()
    
    one_word_place = {'USA','US','England','Asia','China','Europe','Africa','Texas','Atlantic','Japan',
                     'Singapore','Koera','Germany','France','North','East','West','South'}
    if first_name in one_word_place or last_name in one_word_place:
        return False
    two_word_place = {'Los Angeles','San Francisco','United Kingdom','Wall Street','New York','Hong Kong',
                     'Silicon Valley'}
    if name in two_word_place:
        return False
    Financial = {'Economy','Economist','Economics','Federal','Management','Trading','Finance','Financial','Money'
              ,'Debt','Balance','Environment'}
    if first_name or last_name in Financial:
        return False
    policy = {'House','Government','Congress','Senator','Office','President','Budget','Policy',
                'Congressman','Congresswoman','Majority','Minority','Politics'}
    if first_name or last_name in policy:
        return False
    organisation = {'Corp','Inc','Ltd','Corporation','School','University','Bureau'}
    if first_name or last_name in organisation:
        return False
    
    return True

In [34]:
def find_names(article):

    names = map(lambda x : re.findall(r'[A-Z][a-z]* [A-Z][a-z]*',x),article)
    names = filter(None, names)
    names = list(itertools.chain(*names))

    
    names = filter(check_sw, names)
    names = filter(check_notceo,names)
    return names

In [35]:
from nltk.tokenize import TweetTokenizer

In [36]:
def get_features(names, article):
    
    features =[]
    
#     def find_which_sentence(name, sentence):
#         return ((" " + name + " ") in sentence or (" " + name == sentence[-len(name)+1:]) or (name + " " == sentence[:len(name)+1]))
    
    def check_in_sentence(name, sentence):
        
        return ((" " + name + " " in sentence)
             or (sentence[:len(name)+1] == name + " ")
             or (sentence[-len(name)+1:] == " " + name))
   
    for name in names:
        sentences = filter(lambda x: check_in_sentence(name, x), article)
        for sentence in sentences:
            
            
            feature = { "name" : name }
            
            first_name, last_name = name.split(" ")
            feature['first_name_length'] = len(first_name)
            feature['last_name_length'] = len(last_name)
            
            s = re.sub(r"[^A-Za-z0-9 ]", " ", sentence)
            tk = TweetTokenizer()
            words = tk.tokenize(s)
            
            feature['first_name_position'] = words.index(first_name)
            
            
            pos_words = nltk.pos_tag(words)
            feature['first_name_pos'] = pos_words[words.index(first_name)][1]
            feature['last_name_pos'] = pos_words[words.index(last_name)][1]
            
            before_word_index = words.index(first_name)-1
            after_word_index = words.index(last_name) + 1
            
            if before_word_index < 0:
                feature['before_word_Cap'] = 0
                feature['before_word_Pos'] = 'NULL'
            else:
                feature['before_word_Cap'] = int(words[before_word_index].istitle())
                feature['before_word_Pos'] = pos_words[before_word_index][1]
            
            if after_word_index >= len(words):
                feature['after_word_Cap'] = 0
                feature['after_word_Pos'] = 'NULL'
            else:
                feature['after_word_Cap']=int(words[after_word_index].istitle())
                feature['after_word_Pos'] = pos_words[after_word_index][1]

            
            
            feature['is_ceo'] = int(name in ceo_name)
            
            features.append(feature)
    return features
            

In [37]:
import pandas as pd
import numpy as np

In [38]:
train =[]
for i, article in enumerate(train_set):
    names = find_names(article)
#     print(list(names))
    features = get_features(names,article)
    
    train.extend(features)
    
    if i % 500 == 0: 
        print(f"This program has processed {i} articles")
    
df_train = pd.DataFrame(train)

This program has processed 0 articles
This program has processed 500 articles
This program has processed 1000 articles
This program has processed 1500 articles
This program has processed 2000 articles
This program has processed 2500 articles
This program has processed 3000 articles
This program has processed 3500 articles
This program has processed 4000 articles
This program has processed 4500 articles
This program has processed 5000 articles
This program has processed 5500 articles
This program has processed 6000 articles
This program has processed 6500 articles
This program has processed 7000 articles
This program has processed 7500 articles
This program has processed 8000 articles
This program has processed 8500 articles
This program has processed 9000 articles
This program has processed 9500 articles
This program has processed 10000 articles
This program has processed 10500 articles
This program has processed 11000 articles
This program has processed 11500 articles
This program has

In [24]:
train[0]

NameError: name 'train' is not defined

In [391]:
import sklearn
from sklearn import metrics
from sklearn.utils import resample
from sklearn.ensemble import RandomForestClassifier
from sklearn import tree

In [392]:
df_train['is_ceo'].value_counts()

0    147443
1      8348
Name: is_ceo, dtype: int64

In [393]:
df_positive = df_train[df_train.is_ceo==1]
df_negative = df_train[df_train.is_ceo==0]
resample_class = len(df_positive) * 10
df_negative = resample(df_negative, replace = False, n_samples = resample_class)
df_positive = resample(df_positive, replace = True, n_samples = resample_class)
df_train2 = pd.concat([df_negative, df_positive])
df_train2.after_word_Pos = pd.factorize(df_train2.after_word_Pos)[0]
df_train2.before_word_Pos = pd.factorize(df_train2.after_word_Pos)[0]
df_train2.first_name_pos = pd.factorize(df_train2.first_name_pos)[0]
df_train2.last_name_pos = pd.factorize(df_train2.last_name_pos)[0]

In [394]:
RFC = RandomForestClassifier(n_estimators = 20)
featuress = [i for i in df_train2.columns if i not in ("is_ceo", "name")]
RFC.fit(df_train2[featuress], df_train2["is_ceo"])

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=20, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [398]:
validation= [ ]
for i, article in enumerate(validating_set):
    names = find_names(article)
    features = get_features(names, article)
    validation.extend(features)
    
    if i % 1000 == 0: print(f"The program has processed {i} articles")
    
dfv = pd.DataFrame(validation)

Processed 0 articles
Processed 1000 articles
Processed 2000 articles
Processed 3000 articles
Processed 4000 articles
Processed 5000 articles
Processed 6000 articles
Processed 7000 articles


In [399]:
len(validating_set)

7180

In [401]:
dfv.after_word_Pos = pd.factorize(dfv.after_word_Pos)[0]
dfv.before_word_Pos = pd.factorize(dfv.after_word_Pos)[0]
dfv.first_name_pos = pd.factorize(dfv.first_name_pos)[0]
dfv.last_name_pos = pd.factorize(dfv.last_name_pos)[0]

In [403]:
def get_metrics(truth, predicted):
    
    
    confusion_matrix = metrics.confusion_matrix(truth, predicted)
    accuracy = metrics.accuracy_score(truth, predicted)
    precision = metrics.precision_score(truth, predicted)
    recall = metrics.recall_score(truth, predicted)
#     F1 = metrics.f1_score(truth, predicted)
    
    print(f"Confusion Matrix:\n {confusion_matrix}\n")
    print(f"Accuracy: {accuracy}")
    print(f"Precision: {precision}")
    print(f"Recall: {recall}")
    print(f"F1 Score: {F1}")
    
    return

In [407]:
predict = RFC.predict(dfv[featuress])
get_metrics(dfv.is_ceo, predict)

Confusion Matrix:
 [[49985  4508]
 [ 1363   851]]

Accuracy: 0.8964678082070997
Precision: 0.15879828326180256
Recall: 0.3843721770551039
F1 Score: 0.22474580747392048


In [412]:
test = [ ]
for i, article in enumerate(test_set):
    names = find_names(article)
    features = get_features(names, article)
    test.extend(features)
    
    if i % 1000 == 0: print(f"The program has processed {i} articles")
    
dftest = pd.DataFrame(test)
dftest.after_word_Pos = pd.factorize(dftest.after_word_Pos)[0]
dftest.before_word_Pos = pd.factorize(dftest.after_word_Pos)[0]
dftest.first_name_pos = pd.factorize(dftest.first_name_pos)[0]
dftest.last_name_pos = pd.factorize(dftest.last_name_pos)[0]

The program has processed 0 articles
The program has processed 1000 articles
The program has processed 2000 articles
The program has processed 3000 articles
The program has processed 4000 articles
The program has processed 5000 articles
The program has processed 6000 articles
The program has processed 7000 articles


In [413]:
predict = RFC.predict(dftest[featuress])
get_metrics(dftest.is_ceo, predict)

Confusion Matrix:
 [[46697  5786]
 [ 1659   989]]

Accuracy: 0.8649580091055848
Precision: 0.14597785977859778
Recall: 0.3734894259818731
F1 Score: 0.20991191764830733


In [414]:
predt = RFC.predict(df_train2[featuress])
predv = RFC.predict(dfv[featuress])
preds = RFC.predict(dftest[featuress])


found_ceos = [ ]
found_ceos.extend(list(df_train2.iloc[np.where(predt == 1)].name))
found_ceos.extend(list(dfv.iloc[np.where(predv == 1)].name))
found_ceos.extend(list(dftest.iloc[np.where(preds == 1)].name))


with open("found_ceos.txt", "w") as f:
    for ceo in found_ceos:
        f.write(ceo + "\n")

                                            2. Extracting percent 